In [45]:
#System Imports
import os
import glob
import dill as pickle


#Math Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import sklearn as sk

#Data Analysis Imports
import scipy.io
import librosa
import librosa.display
import pandas as pd
from pandas import DataFrame

#NN Import
import tensorflow as tf

import utils
util = utils.utils_cls()



tf.reset_default_graph()

In [5]:
def get_directories(ROOT_PATH, directory):

    directories = [d for d in os.listdir(directory)
                if os.path.isdir(os.path.join(directory, d))]

    return directories

In [9]:

def lstm_cell(n_hidden,state_is_tuple = True):
  return tf.contrib.rnn.BasicLSTMCell(n_hidden)

In [54]:

def RNN(x, weight, bias, number_of_layers, n_hidden):
    
    init_state = tf.placeholder(tf.float32, [number_of_layers, 2, batch_size, n_hidden])

    state_per_layer_list = tf.unstack(init_state, axis=0)
    rnn_tuple_state = tuple(
    [tf.nn.rnn_cell.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1])
     for idx in range(number_of_layers)]
)
    
    cell = tf.nn.rnn_cell.LSTMCell(n_hidden, state_is_tuple=True)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell] * number_of_layers, state_is_tuple=True)
    states_series, current_state = tf.nn.dynamic_rnn(cell, tf.expand_dims(x, -1), initial_state=rnn_tuple_state)
    states_series = tf.reshape(states_series, [-1, n_hidden])
    #cell = tf.contrib.rnn.MultiRNNCell(
    #    [lstm_cell(n_hidden, state_is_tuple=True) for _ in range(number_of_layers)])
    ##
    #cell = tf.contrib.nn.RNNcell()
    #output, state = tf.nn.static_rnn(cell, x, dtype=tf.float32)
    #output = tf.transpose(output, [1, 0, 2])
    #output = tf.transpose(output, [1, 0, 2])
    #last = tf.gather(output, int(output.get_shape()[0]) - 1)
    return tf.nn.softmax(tf.matmul(states_series, weight) + bias)

In [26]:
def get_data():
    ROOT_PATH = "D:/Masters/Data"

    train_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Training")
    test_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Testing")
    print('train data directory' + str(train_data_directory))
    print('test data directory' + str(test_data_directory))  
    train_directories = get_directories(ROOT_PATH, train_data_directory)
    test_directories = get_directories(ROOT_PATH, test_data_directory)
    print('train  directories' + str(train_directories))

    raw_sounds_tr, sr_tr, tr_labels = util.load_sound_wave(train_data_directory, train_directories, file_ext="*.wav")
    raw_sounds_ts, sr_ts, ts_labels = util.load_sound_wave(test_data_directory, test_directories, file_ext="*.wav")
    return raw_sounds_tr, raw_sounds_ts, sr_tr, sr_ts, tr_labels, ts_labels

In [28]:

raw_sounds_tr, raw_sounds_ts, sr_tr, sr_ts, tr_labels, ts_labels = get_data()


train data directoryD:/Masters/Data\Autoencoder test/Training
test data directoryD:/Masters/Data\Autoencoder test/Testing
train  directories['Noise', 'Minke']


In [31]:
str_raw_sounds_tr = str(raw_sounds_tr)
str_raw_sounds_ts = str(raw_sounds_ts)


print(str_raw_sounds_ts)

[-0.01684551 -0.01755317 -0.01826035 ..., -0.00663429 -0.00532464  0.        ]


In [47]:
training_file = "D:\Masters\Data/Autoencoder test/Training/training_tester.pkl"
util.DILL_pickle_files(raw_sounds_tr, training_file)
testing_file = "D:\Masters\Data/Autoencoder test/Testing/testing_tester.pkl"
util.DILL_pickle_files(raw_sounds_ts, testing_file)





TypeError: DILL_pickle_files() takes 2 positional arguments but 3 were given

In [34]:
pickled_sounds_tr = util.DILL_unpickle_files(training_file)

pickled_sounds_ts = util.DILL_unpickle_files(testing_file)

In [48]:


sounds_tr = tf.convert_to_tensor(pickled_sounds_tr, dtype=tf.float32, name='sounds_tr' )
sounds_ts = tf.convert_to_tensor(pickled_sounds_ts, dtype=tf.float32, name='sounds_ts')


print(sounds_tr)

Tensor("sounds_tr:0", shape=(897921,), dtype=float32)


In [51]:
    def one_hot_encode( labels):
        n_labels = labels
        n_unique_labels = len(np.unique(labels))
        one_hot_encode = np.eye(n_unique_labels)

        return one_hot_encode


In [55]:
#def main():
"""

:return:
"""


#sounds_ts = tf.convert_to_tensor(raw_sounds_ts, dtype=tf.float32, name='sounds_ts')
######################################
tr_labels = one_hot_encode(tr_labels)
ts_labels = one_hot_encode(ts_labels)



learning_rate = 0.01
training_iters = 1000
batch_size = 50
display_step = 200

# Network Parameters

number_of_layers = 2

n_hidden = 300
n_classes = 2

x = tf.placeholder(tf.float32, [None, 100, 1], name='x')
y = tf.placeholder(tf.float32, [None, n_classes], name='y')

weight = tf.Variable(tf.random_normal([n_hidden, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))
#raw_sounds_tr = tf.unstack(raw_sounds_tr)
#raw_sounds_ts = tf.unstack(raw_sounds_ts)

prediction = RNN(x, weight, bias, number_of_layers, n_hidden)

# Define loss and optimizer
loss_f = -tf.reduce_sum(y * tf.log(prediction))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_f)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as session:
    saver = tf.train.Saver()
    session.run(init)

    for itr in range(training_iters):
        offset = (itr * batch_size) % (tr_labels.shape[0] - batch_size)
        batch_x = sounds_tr[offset:(offset + batch_size)]
        print(batch_x)
        batch_x = np.squeeze(batch_x)
        batch_y = tr_labels[offset:(offset + batch_size)]
        _, c = session.run([optimizer, loss_f], feed_dict={x: batch_x, y: batch_y})

        if itr % display_step == 0:
            # Calculate batch accuracy
            acc = session.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = session.run(loss_f, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(epoch) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print('Test accuracy: ', round(session.run(accuracy, feed_dict={x: sounds_ts, y: ts_labels}), 3))



ValueError: Shape (?, 1, 1) must have rank 2